In [1]:
import pandas
import jieba
import numpy
# import sklearn

ModuleNotFoundError: No module named 'sklearn'

In [2]:
!ls

ZhuLangXinSong-BiaoZhun     get_info.py
ZhuLangXinSong-BiaoZhun.zip master.zip
analysis_message.ipynb      resewq.py
analysis_message_done.ipynb stopwords
ask_message.csv             suggest_message.csv
chromedriver                train.py
complain_message.csv


In [6]:
# 读取读取
import pymongo
client = pymongo.MongoClient(host='localhost', port=27017)
db = client.PubOpinionMonitoring
collection = db.ChinaArticle
message = collection.find()

In [7]:
# 数据处理，处理需要的数据
need_data = pandas.DataFrame(message)
            


In [8]:
# 打印数据
print(need_data.tail())
# 打印数据描述
print(need_data.describe())
# 查看数据的维度
print(need_data.shape)
# 查看数据类型
print(type(need_data))


                          _id   author  \
201  5c75401b1453365a02380b5e   作者：研究部   
202  5c75401b1453365a02380b5f      作者：   
203  5c75401c1453365a02380b60  作者： 研究部   
204  5c75401d1453365a02380b61      作者：   
205  5c75401e1453365a02380b62   作者：方倾燃   

                                               content    source  \
201  Li+研究丨开门红！一月装机量4.98GWh  同比增长290.94%-Li+研究-电池中国...  来源：电池中国网   
202  超威集团联手京东为消费者供应50万只汽车蓄电池-财经-电池中国网    日前，超威集团与国内...  来源：电池中国网   
203  Li+研究丨316批新车公示电池配套分析：国轩高科跃居第二 威睿电动/东风海博首进前五-Li...  来源：电池中国网   
204  电池企业或靠网约车打场漂亮仗？-独家观察-电池中国网     2019年，动力电池企业+网约...  来源：电池中国网   
205  拿下国内外一线客户大订单 赢合科技2018年净利润增长近五成-锂电池-电池中国网    2月...  来源：电池中国网   

                         time                                        title  \
201  发布时间：2019-02-18 14:37:33           Li+研究丨开门红！一月装机量4.98GWh 同比增长290.94%   
202  发布时间：2019-02-25 14:45:04                      超威集团联手京东为消费者供应50万只汽车蓄电池   
203  发布时间：2019-02-18 14:34:01  Li+研究丨316批新车公示电池配套分析：国轩高科跃居第二 威睿电动/东风海博首进前五   
204  发布时间：2019-02-18 14:36

In [10]:
need_data[:5]

,_id,author,content,source,time,title,url
0,5c753c5b1453365a02380a95,作者：李超,警笛声拉响，坐在警车上的周林(化名)才缓过神来，他意识到“确实犯错了”。这一天是2018年6...,来源：中国青年报,发布时间：2019-02-26 09:01:41,“错误定位”让创业大学生误入歧途,http://news.china.com.cn/2019-02/26/content_74...
1,5c753c5d1453365a02380a96,作者：鞠焕宗,2月22日晚，张世玉（前右一）与吕新（前右二）在北京南站等候回天津的火车。2018年对于“9...,来源：新华社,发布时间：2019-02-26 09:50:51,“90后”小夫妻的津京生活,http://news.china.com.cn/2019-02/26/content_74...
2,5c753c5d1453365a02380a97,作者：杨毅,大连铁路警方24日发布消息称，近日，一名女性乘客乘火车旅行，在餐车上独自喝了半瓶多白酒后，在...,来源：中国新闻网,发布时间：2019-02-25 08:49:32,大连一女子火车上畅饮 酒后失态袭警被拘留,http://news.china.com.cn/2019-02/25/content_74...
3,5c753c5d1453365a02380a98,作者：佚名,法者，治之端也。2月25日，习近平总书记主持召开中央全面依法治国委员会第二次会议并发表重要讲...,来源：中央广播电视总台央视网,发布时间：2019-02-26 20:22:01,中央全面依法治国委员会新年首会 干货全在这里,http://news.china.com.cn/2019-02/26/content_74...
4,5c753c5d1453365a02380a99,作者：佚名,来设想这样一种情况：你从来没有去注册过公司，可不知什么时候你名下却多了一家甚至几家、几十家公...,来源：央视新闻客户端,发布时间：2019-02-26 08:46:31,男子莫名成公司老板上老赖名单：奔波300多天难解决,http://news.china.com.cn/2019-02/26/content_74...


In [11]:
# 清理数据
# 清理内容字符数小于6的数据
need_data = need_data[need_data['content'].str.len() > 7]

In [12]:
need_data.shape

(193, 7)

In [13]:
# 情感分析
"""
在这部分内容中，我们调用了SnowNLP的情感分析，它是一个python写的类库，可以方便的处理中文文本内容，不用我们实现其中具体的代码。
一般来说，情感分析的目的是为了找出作者观点的态度，是正向还是负向，或者更具体的，我们希望知道他的情绪。
在这里，我们希望了解到好友签名所表达出来的情感是积极的，还是中立、负面的，
比如说在以下例子中，我们对"这个商品我非常喜欢，颜色很合我意！"这句话进行了预处理，并通过训练好的模型预测其的情感。
在这里，我们简单地假设大于0.66表示积极，低于0.33表示消极，其他表示中立。"""

from snownlp import SnowNLP

# #一个demo
# text = "这个商品我非常喜欢，颜色很合我意！"
# sentiment = SnowNLP(text).sentiments
# print(sentiment)


# 用一个列表存储经过情感分析处理的数据
sentiments = []
for i in tList:
    sentiments.append(SnowNLP(i).sentiments) 
    
# 打印数据
print(sentiments)

# 使用一个字典返回情感分析统计的数据
sentiments_dict = {"positive": 0,
                    "neutral": 0,
                    "negative": 0}

for i in sentiments:
    if i > 0.66:
        sentiments_dict["positive"] += 1
    elif 0.33 <= i <= 0.66:
        sentiments_dict["neutral"] += 1
    elif 0.33 > i:
        sentiments_dict["negative"] += 1

print(sentiments_dict)

ModuleNotFoundError: No module named 'snownlp'

In [15]:
###  打印试试看看图片，可以用于前端展示
labels = [u'Negative',u'Neutral',u'Positive']
values = (sentiments_dict["negative"] , entiments_dict["neutral"], sentiments_dict["positive"])
plt.xlabel(u'Sentiment Analysis')
plt.ylabel(u'Number')
plt.xticks(range(3),labels)
plt.bar(range(3), values)

plt.title('Sentiment Analysis of OPA')
plt.show()

NameError: name 'sentiments_dict' is not defined

In [17]:
# 数据转列表 列表才可以使用数据
content = need_data.content.values.tolist()
print(len(content))

193


In [18]:
import random
content[random.randint(1,197)]

'2月19日下午，国务委员兼国防部长魏凤和在八一大楼会见了来访的越南国防部副部长阮志咏。李晓伟摄国务委员兼国防部长魏凤和19日在北京八一大楼会见了来访的越南国防部副部长阮志咏。魏凤和说，中越友谊根基深厚，理想信念相通，关系特殊重要，是具有战略意义的命运共同体。近年来，在习近平总书记和阮富仲总书记共同引领下，中越全面战略合作伙伴关系保持健康稳定发展势头。中国军队愿同越军共同努力，落实好两党两国高层重要共识，加强战略沟通协调，深化各领域交流合作，妥善处理矛盾分歧，相互理解、相互支持、相互帮助，推动两军关系跨上新台阶，为维护两国根本利益、维护地区安全稳定作出积极贡献。阮志咏表示，越共中央、中央军委珍视越中传统友谊，高度重视发展两国两军关系，愿与中方共同努力，进一步深化各领域务实交流合作，推动越中两国两军关系一年更比一年好。军委联合参谋部副参谋长邵元明、越南驻华大使邓明魁等参加会见。（尹航）2月19日下午，国务委员兼国防部长魏凤和在八一大楼会见了来访的越南国防部副部长阮志咏。李晓伟摄'

In [19]:
# 使用分词，把文章分为单独词语
content_s = []
for i in content:
    current_mess = jieba.lcut(i)
    # 判断词语
    if len(current_mess) > 2 and current_mess!='\r\n' and current_mess != ' ': 
        content_s.append(current_mess)
print(len(content_s))

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/mk/b4xd_dcs6_dc714m79rzbw7c0000gn/T/jieba.cache
Loading model cost 0.891 seconds.
Prefix dict has been built succesfully.


193


In [20]:
df_content = pandas.DataFrame({"contene_s":content_s})

In [23]:
# 查看分词之后的情况 随机查看
print(content_s[random.randint(1,193)])

print(len(content_s))

['争夺', '激烈', '！', '340', '亿元', '国内', '市场需求', ' ', '锂电', '设备', '行业', '增速', '达', '30%', '-', '独家', '观察', '-', '电池', '中国', '网', '\xa0', ' ', '\xa0', ' ', '进入', '2019', '年', '以来', '，', '已经', '有', '多家', '锂电', '设备', '企业', '发布', '2018', '年度', '业绩', '预告', '和', '年报', '，', '受益', '于', '锂电池', '行业', '的', '高', '景气', '发展', '，', '赢合', '科技', '、', '先导', '智能', '等', '锂电', '设备', '龙头企业', '成绩', '喜人', '。', '1', '月', '30', '日', '，', '先导', '智能', '发布', '业绩', '预告', '，', '公司', '预计', '2018', '年度', '归属', '上市公司', '股东', '的', '净利润', '为', '6.72', '亿元', '至', '8.33', '亿元', '，', '同比', '增长', '25.00%', '至', '55.00%', '。', '先导', '智能', '表示', '，', '得益于', '锂电池', '行业', '快速', '持续', '发展', '，', '公司', '在', '稳定', '原有', '客户', '的', '基础', '上', '，', '积极', '开拓', '新', '客户', '，', '锂电池', '相关', '设备', '的', '生产', '销售', '业绩', '较', '去年同期', '有', '显著', '提升', '。', '1', '月', '22', '日', '，', '金银', '河', '发布', '2018', '年', '年报', '，', '报告', '期内', '实现', '营业', '收入', '6.42', '亿元', '，', '同比', '增长', '31.03%', '。', '同时', '，', '金银', '河', '还', '发布', '了', '2019', 

In [24]:
# 读取停用词，停用词的使用，是为减少一个常见词语对这个语意的影响
stopwords = pandas.read_csv('stopwords/stop_words_zh.txt',index_col=False, sep="\t", quoting = 3, names=['stopword'], encoding='utf-8')
stopwords.head()

FileNotFoundError: [Errno 2] File b'stopwords/stop_words_zh.txt' does not exist: b'stopwords/stop_words_zh.txt'

In [16]:
# 去掉停用词
def drop_stopword(contents, stopwords):
    content_clean = []
    for line in contents:
        line_clean = []
        for word in line:
            if word in stopwords:
                continue
            line_clean.append(word)            
        content_clean.append(line_clean)
    return content_clean

contents = df_content.contene_s.values.tolist()
stopwords = stopwords.stopword.values.tolist()
content_clean =drop_stopword(contents, stopwords)

In [17]:
df_content_drop = pandas.DataFrame({"content_clean": content_clean, 'label':all_content['label']})
df_content_drop.head()

,content_clean,label
0,"[工商, 审批, 10, 多天, 没, 进展, 不是, 说好, 一到, 2, 工作日]",1
1,"[甘肃, 处处, 落后, 网站, 不, 例外, 经济, 发展, 模式, 不好, 抄, 照搬,...",1
2,"[尊敬, 领导, 之前, 教师, 职称, 评审, 政策, 都, 班主任, 工作, 年限, 不...",1
3,"[政府, 采购, 协议, 供货, 合同, 以前, 采用, 人工, 备案, 财政厅, 需要, ...",1
4,"[甘肃, 公安, 政务, 服务平台, 浏览器, 不, 兼容, 360, 浏览器, 每次, 登...",1


In [19]:
# 使用jieba分析语句中重要的词语，提取一段的关键词，减少阅读量   参数topK 是最后返回数据的类型
import jieba.analyse
index = 80
print(sug_content['content'][index])
content_s_str = "".join(content_s[index])
print(' '.join(jieba.analyse.extract_tags(content_s_str, topK = 5, withWeight = False)))

企业法人注册一直显示实名认证失败，但是个人注册可以，不知道能否改进下，让企业法人注册方便一点，开一个公司不容易，好几天先注册不上，很浪费时间的，麻烦有关领导看见了能不能改进下。
注册 企业法人 改进 不上 浪费时间


In [ ]:
# 处理数据用于词云使用
words_count = df_all_drop.groupby(by=['all_words'])['all_words'].agg({"counts": numpy.size})
words_count = words_count.reset_index().sort_values(by=['counts'], ascending = False)
words_count.head()

In [ ]:
# 画词云
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 5.0)

In [ ]:
wordcloud = WordCloud(font_path= 'ZhuLangXinSong-BiaoZhun/ZhuLangXinSong-BiaoZhun-2.otf',background_color='white', max_font_size=80)
word_frequence = {x[0]:x[1] for x in words_count.head(300).values}
wordcloud = wordcloud.fit_words(word_frequence)
plt.imshow(wordcloud)

使用LDA：主题模型
格式要求：list of list，分词好的整个语料

In [20]:
from gensim import corpora, models, similarities
import gensim

In [21]:
dictionary = corpora.Dictionary(content_clean)
corpus = [dictionary.doc2bow(sentence) for sentence in content_clean]

In [22]:
# 类似Kmeans 
lda = gensim.models.LdaModel(corpus = corpus, id2word = dictionary, num_topics = 3)  


In [23]:
# 1号分类结果
print(lda.print_topic(1,topn=5))

0.015*"月" + 0.014*"办理" + 0.013*"年" + 0.009*"日" + 0.008*"需要"


In [25]:
for topic in lda.print_topics(num_topics = 3, num_words = 5):
    print(topic[1])

0.011*"年" + 0.010*"不" + 0.008*"月" + 0.008*"办理" + 0.006*"没有"
0.015*" " + 0.009*"办理" + 0.009*"申请" + 0.008*"需要" + 0.007*"没有"
0.016*"办理" + 0.012*"公司" + 0.011*"不" + 0.011*"名称" + 0.010*"申请"


In [24]:
# 打印有多少个分类
all_content.label.unique()


array([1, 2, 3])

In [66]:
# 本来的标签值就是为123，所有不需要做标签值的转变
# # 把lable的值换成是数字
# label_mapping = {"建议":1, "咨询":2, "投诉":3}
# all_content['label'] = all_content['label'].map(label_mapping)
# all_content.head()

In [25]:
print(df_content_drop['content_clean'].shape)
print(all_content.shape)

(2655,)
(2655, 2)


In [26]:
df_content_drop.head()

,content_clean,label
0,"[工商, 审批, 10, 多天, 没, 进展, 不是, 说好, 一到, 2, 工作日]",1
1,"[甘肃, 处处, 落后, 网站, 不, 例外, 经济, 发展, 模式, 不好, 抄, 照搬,...",1
2,"[尊敬, 领导, 之前, 教师, 职称, 评审, 政策, 都, 班主任, 工作, 年限, 不...",1
3,"[政府, 采购, 协议, 供货, 合同, 以前, 采用, 人工, 备案, 财政厅, 需要, ...",1
4,"[甘肃, 公安, 政务, 服务平台, 浏览器, 不, 兼容, 360, 浏览器, 每次, 登...",1


In [27]:
# 切分数据  训练集和测试集默认分为3/1  test_size= 0~1
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_content_drop['content_clean'].values, df_content_drop['label'].values, random_state = 1, shuffle = True)



In [28]:
#打印训练集大小
print(x_train[1])
x_train.shape

['个体户', '名称', '申请', '审核', '都', '好', '几天', '没有', '一点', '进展', '现在', '不是', '简化', '程序', '还', '慢']


(1991,)

In [29]:
# 列表转字符串
words = []
for line_index in range(len(x_train)):
    try:
        words.append(' '.join(x_train[line_index]))
    except IndexError:
        print(line_index, words.index)
print(words[0])
                     

您好 2013 年 甘肃农业大学 取得 农业 推广 硕士学位 当时 报考 在职 研究生 没有 毕业证 学位证 不 知道 报考


In [30]:
print(len(words))

1991


In [31]:
# 测试词语转向量
from sklearn.feature_extraction.text import CountVectorizer
texts = ['dog cat fish', 'dog cat cat', 'fish, bird', 'bird','bird cat']
cv = CountVectorizer()
cv_fit = cv.fit_transform(texts)

# 打印结果
# 打印所有的词语
print(cv.get_feature_names())
# 打印词语占的向量
print(cv_fit.toarray())
# 打印合计
print(cv_fit.toarray().sum(axis = 0))

['bird', 'cat', 'dog', 'fish']
[[0 1 1 1]
 [0 2 1 0]
 [1 0 0 1]
 [1 0 0 0]
 [1 1 0 0]]
[3 4 2 2]


In [32]:
# 词语转向量 设置转换规则 计数
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(analyzer='word', max_features=5000, lowercase = False)
vec.fit(words)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [33]:
# 训练
from sklearn.naive_bayes import MultinomialNB

classifer = MultinomialNB()
classifer.fit(vec.transform(words), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [34]:
# 测试集数据处理
test_words = []
for line_index in range(len(x_test)):
    try:
        test_words.append(' '.join(x_test[line_index]))
    except:
        print(line_index, word_index)
test_words[0]
        

'你好 政务网 企业 登录 后 需要 进行 企业 信息 变更 出现 确认 企业 信息 窗口 工商 密码 填写 告知 谢谢'

In [35]:
# 打印准确率
classifer.score(vec.transform(test_words), y_test)

0.8765060240963856

 方法2 tf词频使用的

In [36]:
# 使用tf词频
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word', max_features = 5000, lowercase = False)
vectorizer.fit(words)
# 打印结果
# 训练
from sklearn.naive_bayes import MultinomialNB
classifer = MultinomialNB()
classifer.fit(vectorizer.transform(words), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [37]:
# 打印准确率
classifer.score(vectorizer.transform(test_words), y_test)

0.8358433734939759

In [118]:
# # 验证数据
# perdict_data = cv.fit_transform(x_train[1])
# perdict_data
# classifer.predict(perdict_data)
# classifer.predict(vec.transform(x_train[1882]))[1]

2

In [38]:
# 验证某条数据
new_str = '甘肃省省领导： 我公司景泰长城冶炼有限责任公司，在2016年电力用户和发电企业的直购电交易上'
perdict_str = jieba.lcut(new_str)

In [39]:
# 定义去掉停用词函数
def drop_stopword_peridict(contents, stopwords):
    content_clean = []
    for word in contents:
        if word in stopwords:
            continue
        content_clean.append(word)            
    return content_clean

In [40]:
# 去掉停用词
peridict_clean =drop_stopword_peridict(perdict_str, stopwords)

In [43]:
classifer_dict = {1: "建议", 2: "咨询",3: "投诉"}
classifer_dict[classifer.predict(vec.transform(peridict_clean))[1]]

'咨询'

In [ ]:
#  因为咨询的数据量过大，在这里里面成为了主要的分类。 模型的正确率不是很高， 可以有两个地方提高正确率，
# 1.数据采集，需要均衡三类数据，
# 2.停用词，需要对具体情况进行，因为时间的原因，我在网上找了很多的停用词来使用。